<a href="https://colab.research.google.com/github/silverstar0727/1day-1commit-challenge/blob/master/bitcoin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# google drive와 연동
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
# 연동 테스트 코드
with open('/content/gdrive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')

!cat /content/gdrive/My\ Drive/foo.txt

Hello Google Drive!

In [6]:
import numpy as np
import pandas as pd


df = pd.read_csv("/content/gdrive/My Drive/BTC-USD.csv", names = ['time', 'low', 'high', 'open', 'close', 'volume'])
print(df.head())

         time          low         high         open        close    volume
0  1528968660  6489.549805  6489.560059  6489.560059  6489.549805  0.587100
1  1528968720  6487.370117  6489.560059  6489.549805  6487.379883  7.706374
2  1528968780  6479.410156  6487.370117  6487.370117  6479.410156  3.088252
3  1528968840  6479.410156  6479.419922  6479.419922  6479.410156  1.404100
4  1528968900  6475.930176  6479.979980  6479.410156  6479.979980  0.753000


In [0]:
# 데이터를 넣을 빈 데이터프레임을 만듦
main_df = pd.DataFrame()

In [0]:
#파라미터를 변수로 설정
SEQ_LEN = 60 # 몇분짜리를 한 덩어리로 학습시킬건지 (ex 60분)
FUTURE_PERIOD_PREDICT = 3 #몇 분 후 가격을 기준으로 등락을 결정할 건지 (3분 후)
RATIO_TO_PREDICT = "BTC-USD" # 어느 코인 가격을 예측?

In [10]:
ratios = ["BTC-USD", "BTC-USD", "ETH-USD"]
for ratio in ratios:
    dataset = f'training_datas/{ratio}.csv'
    df = pd.read_csv(dataset, names=['time','low','high','open','close','volume'])

    df.rename(columns={"close": f"{ratio}_close", "volume" : f"{ratio}_volume"}, inplace=True)


    # 일단 time을 index로 두고, 컬럼은 종가와 거래량만 남김
    df.set_index("time", inplace=True)
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]
    # 그리고 이것들을 join한다 -> 여러 코인 데이터를 하나의 df에 join
    if len(main_df)==0:
        main_df = df
    else:
        main_df = main_df.join(df)

main_df.fillna(method="ffill",inplace=True)
main_df.dropna(inplace=True)
print(main_df.head())


## target값,즉 등락을 표시하기 위한 함수
def classify(current, future): #현재와 미래 가격을 인자로 받아서
    if float(future) > float(current):
        return 1 # 오르면 1
    else:
        return 0 # 떨어지면 0

#여기서 미래의 가격이란 3분 후의 가격
main_df['future'] = main_df[f'{RATIO_TO_PREDICT}_close'].shift(-FUTURE_PERIOD_PREDICT)
#그래서 위의 classfy 함수를 돌려 target value를 구한다
main_df['target'] = list(map(classify, main_df[f'{RATIO_TO_PREDICT}_close'], main_df['future']))


# test train split
#시간순으로 마지막 5%에 해당하는 데이터를 테스트 데이터로 쓴다
times = sorted(main_df.index.values) # get the times
last_5pct = sorted(main_df.index.values)[-int(0.05*len(times))]

# validation_main_df는 마지막 5% 이내 시간
validation_main_df = main_df[(main_df.index >= last_5pct)]
# main_df는 앞의 95% 시간에 해당하는 데이터가 들어감
main_df = main_df[(main_df.index < last_5pct)]

FileNotFoundError: ignored